# HST WFC3 PSF Modeling

**Author:** Mitchell Revalski

**Updated:** August 11, 2021

Copyright (c) 2021, Mitchell Revalski

All rights reserved. This source code is licensed under the BSD-style license found in the LICENSE file in the root directory of this source tree.

***

## Introduction

This Jupyter Notebook was created by Dr. Mitchell Revalski to generate Point Spread Function (PSF) models for Hubble Space Telescope (HST) Wide Field Camera 3 (WFC3) data. The user may select from two types of PSF modeling methodologies depending on their science goals. The two modeling options are:

**Stellar:** The code will generate a PSF model by stacking stars extracted from a drizzled science image. This requires a Source Extractor type catalog, or a list of (x,y) coordinates, that correspond to the stars the user would like to stack. The user may set a variety of selection criteria and manually exclude objects.

**Empirical:** The code will generate a PSF model by stacking STScI PSFs from a drizzle created using Varun Bajaj's *wfc3tools* make_model_star_image(). This requires a list of PSF coordinates generated using make_model_star_image(), or a list of (x,y) coordinates corresponding to the model positions in the drizzle.

See: [https://github.com/Vb2341/wfc3tools](https://github.com/Vb2341/wfc3tools)  and  [ISR WFC3 2016-12: Empirical Models for the WFC3/IR PSF - Jay Anderson - Mar 17, 2016](https://www.stsci.edu/files/live/sites/www/files/home/hst/instrumentation/wfc3/documentation/instrument-science-reports-isrs/_documents/2016/WFC3-2016-12.pdf)

The second decision is for the user to select whether to create a PSF model of the entire image, or create independent PSF models of the inner and outer portions of their science image. This is useful for mosaics with a longer effective exposure time in the center and a shorter exposure time near the edges.

Please send questions, comments, and suggestions to [Mitchell Revalski](https://www.mitchellrevalski.com). Thank you, and have a nice day!

***

## Software

The Python environment used to run this notebook was built using conda:<br>
<code>
(base) ...:...$ conda create -n ac3 stsci
</code>

The jupyter notebook can then be run using the following call sequence:<br>
<code>
(base) ...:...$ conda activate ac3
(ac3)  ...:...$ jupyter notebook
</code>

The critical package versions that the code was tested with include:<br>
<code>
python -> 3.7.4
matplotlib -> 3.1.1
numpy -> 1.17.
scipy -> 1.3.1
astropy -> 3.2.1
photutils -> 0.7
</code>    

***

## Table of Contents <a class="anchor" id="tag0"><a>


**&emsp; 1) [Setup Options: Inputs/Outputs](#setup)<br>**
**&emsp; 2) [Setup Options: Filter/PSF Type](#setup2)<br>**
**&emsp; 3) [Setup Options: Model Fitting](#setup3)<br>**
**&emsp; 4) [Setup Options: PSF Positions](#setup4)<br>**
**&emsp; 5) [Setup Options: Source Extractor](#setup5)<br>**
**&emsp; 6) [Setup Options: Manual Selection](#setup6)<br>**
**&emsp; 7) [Start of Main Code](#start)<br>**
**&emsp; 8) [Identify Stars in Image](#select)<br>**
**&emsp; 9) [Select the Best Stars](#clean)<br>**
**&ensp; 10) [Import Science Drizzle](#import)<br>**
**&ensp; 11) [Import Empirical Drizzle](#import2)<br>**
**&ensp; 12) [Extract Subimages](#extract)<br>**
**&ensp; 13) [Define Median Function](#meanfunction)<br>**
**&ensp; 14) [Define Fitting Function](#fitfunction)<br>**
**&ensp; 15) [Image Interpolation](#interpolate)<br>**
**&ensp; 16) [Image Profile Fitting](#fitting)<br>**
**&ensp; 17) [Calculate Image Shifts](#shifts)<br>**
**&ensp; 18) [Align the Images](#align)<br>**
**&ensp; 19) [Median Image Stack](#combine)<br>**
**&ensp; 20) [Reinterpolate Stack](#reinterpolate)<br>**
**&ensp; 21) [Flux Conservation](#check)<br>**
**&ensp; 22) [Measure PSF FWHM](#fwhm)<br>**
**&ensp; 23) [Trouble Shooting](#trouble)<br>**
**&ensp; 24) [Developer Notes](#developer)<br>**
    
***
    
## Imports

The following packages are required to run the Jupyter Notebook:
 - *os* - change and make directories
 - *sys* - set paths to external codes
 - *matplotlib.pyplot* - create graphics
 - *numpy* - math and array functions
      - *unravel_index* - find peak location
 - *operator* - greater and less than functions
 - *scipy* - image interpolation and shifting
      - *interpolate.interp2d* - interpolate arrays
      - *ndimage* - shift interpolated arrays
 - *astropy* - model fitting and file handling
      - *io.fits* - import FITS files
      - *modeling.models* - Gaussian and Moffat models
      - *modeling.fitting* - fitting methods for models
 - *photutils* - peak finder for empirical modeling
      - *detection.find_peaks* - find peaks in *wfc3tools* drizzle
 - *itertools* - iteration tools for loops       
      - *chain* - separate concatenated lists in parallel mode
 - *multiprocessing* - allows for models to be fit using parallel mode
      - *pool* - parallel processing manager that parses jobs
 - *fit_profile_parallel* - same as internally defined *fit_profile()* for parallel processing

The external function packages are documented at the following links:

[https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp2d.html](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp2d.html)

[https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.shift.html](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.shift.html)

[https://docs.astropy.org/en/stable/api/astropy.modeling.functional_models.Gaussian2D.html](https://docs.astropy.org/en/stable/api/astropy.modeling.functional_models.Gaussian2D.html)

[https://docs.astropy.org/en/stable/api/astropy.modeling.functional_models.Moffat2D.html](https://docs.astropy.org/en/stable/api/astropy.modeling.functional_models.Moffat2D.html)

[https://photutils.readthedocs.io/en/stable/api/photutils.detection.find_peaks.html#photutils.detection.find_peaks](https://photutils.readthedocs.io/en/stable/api/photutils.detection.find_peaks.html#photutils.detection.find_peaks)

[https://docs.python.org/3/library/itertools.html](https://docs.python.org/3/library/itertools.html)

[https://docs.python.org/3/library/multiprocessing.html](https://docs.python.org/3/library/multiprocessing.html)

In [ ]:
import os
import sys
import matplotlib.pyplot
import numpy
from numpy import unravel_index
import operator
import scipy
from scipy.interpolate import interp2d
from scipy import ndimage
import astropy
from astropy.io import fits
from astropy.modeling import models
from astropy.modeling import fitting
from photutils import find_peaks
from itertools import chain

## Setup Options: Inputs/Outputs  <a class="anchor" id="setup"><a>
[Table of Contents](#tag0)
    
***

**The user is REQUIRED to specify the following:**
 - <font color=blue>*main_directory*</font> - directory where the code is located.
 
 
 - <font color=blue>*export_name*</font> - the prefix for exported files (only used if *export_results = True*).
 
 
 - <font color=blue>*export_results*</font> - True or False.
      - True: the code will export the PSF models as FITS files.
      - False: the code will run but not output any model files.
 
 
 - <font color=blue>*speak*</font> - True or False.
      - True: the code will verbally report when 1) interpolation, 2) profile fitting, and 3) modeling are complete.
      - False: the code will run silently.
      
      
 - <font color=blue>*print_fit*</font> - True or False.
      - True: the code will print detailed information about the model fits.
      - False: the code will only print error messages about the model fits.
      
      
 - <font color=blue>*parallel_mode*</font> - True or False (only used if *use_peaks = False*).
      - True: the code will run faster by fitting models in parallel (recommended).
      - False: the code will complete the model fits one at a time.
      
      
 - <font color=blue>*number_of_cores*</font> - number of computer cores to employ (only used if *parallel_mode = True*).
      - Value: the integer number of cores to utilize (e.g. 2, 4, 6... recommend half of total cores).
      - 'max': the code will determine the number of computer cores available and use them all.   

In [ ]:
main_directory = '/Users/path/to/code/'

export_name = 'hst_f140w_psf' # Name based on your filter.

export_results = True

speak = True

print_fit = False

parallel_mode = True

number_of_cores = 8

## Setup Options: Filter/PSF Type  <a class="anchor" id="setup2"><a>
[Table of Contents](#tag0)
    
***

**The user is REQUIRED to specify the following:**
 - <font color=blue>*my_filter*</font> - filter of your science image (string, e.g. 'f125w', 'f140w', 'f336w').
 
 
 - <font color=blue>*science_drizzle*</font> - filename of your science image (string, expects build=False for drizzles, should handle both cases).
    
    
 - <font color=blue>*export_directory*</font> - the folder within *../my_filter/results/* where the data products will be saved (string).    
    
 
 - <font color=blue>*rpix*</font> - the half-width dimension of the desired PSF model in pixels (e.g. *rpix* = 34 for a 68x68 pixel model and *rpix* = 34.5 for a 69x69 pixel model).
 
 
 - <font color=blue>*psf_type*</font> - 'stellar' or 'empirical'.
      - 'stellar': assumes the *science_drizzle* is the ouput of the astrodrizzle software.    
      - 'empirical': assumes the *science_drizzle* is the output of Varun Bajaj's *wfc3tools*.
      
      
 - <font color=blue>*dual_psf*</font> - True or False.
      - True: the code will calculate a PSF model for either the 'inner' or 'outer' regions of an image as defined by the *radius_switch* command.
           - Useful for drizzles with long exposure center and short exposure edges to create independent PSF models.
      - False: the code will calculate a single PSF model for the entire image.
      
      
 - <font color=blue>*radius_switch*</font> - options are 'inner' or 'outer' (string, only used if *dual_psf = True*).
      - 'inner': PSF model is calculated using stars INSIDE a circle of radius *rmax*.
      - 'outer': PSF model is calculated using stars OUTSIDE a circle of radius *rmax*.
      
      
 - <font color=blue>*rmax*</font> - radius of a circle (in pixels) that delineates the 'inner' and 'outer' regions of the image.

      
A visual description of the *dual_psf*, *radius_switch*, and *rmax* options are shown below:
![image](docs/dual_psf.jpg)

In [ ]:
my_filter = 'f140w'

science_drizzle = 'hst_f140w_drz_sci.fits'

export_directory = 'v01'

rpix = 34.5

psf_type = 'stellar'

#psf_type = 'empirical'

dual_psf = True

#dual_psf = False

radius_switch = 'inner'

#radius_switch = 'outer'

rmax = 1000.0

This cell determines whether your PSF is even or odd in size and fixes the center pixel location for your PSF model. These lines should only be modified in special cases where an off-centered PSF model is desired or if your model is not centered based on the conventions of your next analysis codes (e.g. indices starting at 0 versus 1). By default the offsets are programmed for SAO DS9 image coordinates: [https://sites.google.com/cfa.harvard.edu/saoimageds9](https://sites.google.com/cfa.harvard.edu/saoimageds9). The user-supplied value of *rpix* is converted to an integer in order to work in array space and extract postage-stamps of the objects. The actual decimal value will be used later to set the size of the PSF model and is encapsulated in the *new_center* and *subimg_shape* variables.

In [ ]:
if rpix % 2 == 0:
    subimg_shape = 'even'
    new_center = rpix
else:
    subimg_shape = 'odd'
    new_center = rpix-1.0
    
rpix = numpy.int(numpy.floor(rpix))

## Setup Options: Model Fitting  <a class="anchor" id="setup3"><a>
[Table of Contents](#tag0)
    
***

**The user is REQUIRED to specify the following:**
 - <font color=blue>*interp_grid_steps*</font> - the interpolation grid resolution (e.g. 10 = 0.1 pixel interpolation scale).
      - Use only even numbers, recommend a value of 2 for testing and 10 or 20 for final results.
      
      
 - <font color=blue>*fit_type*</font> - 'gaussian' or 'moffat'.
      - 'gaussian': the code will fit a Gaussian profile to each object.
      - 'moffat': the code will fit a Moffat profile to each object.
           - A Moffat profile is recommended for fitting stars due to their extended wings.
      
      
 - <font color=blue>*max_iterations*</font> - the maximum number of convergence iterations for the fitting routine.
      - Do NOT set below 100, recommend 2,000+ for IR images and 150,000+ for UVIS images.
      - If the code reports a possible issue with fitting, then increase the number of *max_iterations*.      


 - <font color=blue>*scale_stars*</font> - True or False.
      - True: The flux of each star or model will be scaled to unity before stacking (strongly recommended for stars).
      - False: The flux of each model or star is unaltered and will be mean/median combined (special cases).
 
 
 - <font color=blue>*scale_image*</font> - True or False.
      - True: The total area of the final PSF model will be scaled to unity (strongly recommended).
      - False: The total area of the final PSF model is unaltered after mean and median combining (special cases).  

In [ ]:
interp_grid_steps = 20.

fit_type = 'moffat'

max_iterations = 200000

scale_stars = True

scale_image = True

## Setup Options: PSF Positions  <a class="anchor" id="setup4"><a>
[Table of Contents](#tag0)
    
***

**The user is REQUIRED to specify the following:**
 
 - <font color=blue>*read_positions*</font>   - True or False.
      - True: the initial guess (x,y) coordinates of the PSF centroids will be read from an external text file (see *positions_file* below).
          - This default feature is useful if you already know the approximate centroid locations, or want to use specific stars in an image.      
      - False: the initial guess (x,y) coordinates will be found using a peak finding algorithm (see warning below).    
          - This will select all peaks in the image, so is only recommended for *psf_type = 'empirical'*, where the image contains well-spaced and normalized point sources. To adjust the threshold settings of this function, go to [*peaks_table = find_peaks(...)*](#import2).
    
    
 - <font color=blue>*lock_positions*</font>   - True or False (only used if *read_positions = True*).
      - True: this will override the codes fitting, and use the *exact* (x,y) centroid coordinates from the external file (<font color=red>not recommended</font>).
          - This feature is useful if you already know the exact centroid locations, or for debugging.
      - False: the (x,y) coordinates from the file are used as initial guesses by the fitting routine.
          - This is the recommended default setting, which allows the code to fit Moffat or gaussian profiles to the sources.
      
      
 - <font color=blue>*positions_file*</font>   - a catalog or file of the approximate object locations in the image (only used if *read_positions = True*).
      - This file should contain the stellar or empirical PSF locations, typically a Source Extractor catalog or a two-column text file of (x,y) locations.
          - The type of file is specified by the *use_source_catalog_xy_coords* and *use_list_of_manual_xy_coords* commands below.      
      - If using Varun Bajaj's PSF drizzling tool, then this file may be generated by saving the output of the *generate_input_coordinates()* function.
          - Only use this option with the output from *generate_input_coordinates()* if *lock_positions = False*, as the locations are not distortion corrected.
      

 - <font color=blue>*use_peaks*</font>   - True or False.
      - True: (x,y) coordinates of the PSF centroids are the locations of the peak fluxes.
          - This command may be used when *read_positions* is True or False. It is particularly useful for symmetric PSFs in the UV, where the peak flux location, and the centroid from a Moffat fit profile, are very similar. In this case, the code runs much faster by simply using the peak value.      
      - False: (x,y) coordinates of the PSF centroids will be fit using the profile specified by *fit_type*.
          - This is the default, which allows the code to fit gaussian or Moffat profiles to the objects.

<font color=red>Note that only one of the following two options should be set to True, the other MUST be False:</font>
          
 - <font color=blue>*use_source_catalog_xy_coords*</font>   - True or False (only used if *read_positions = True*).
      - True: the (x,y) coordinates of objects are expected to come from a Source Extractor catalog.
          - Note: the catalog can be made by any tool, e.g. photutils, but must contain the Source Extractor equivalent measures of: ID, X, Y, class_star, mag_iso, and elongation. See the *source_extractor_catalog* variable below for a more detailed description.
      
      - False: set to False if not using a source catalog.     
      
          
 - <font color=blue>*use_list_of_manual_xy_coords*</font>   - True or False (only used if *read_positions = True*).
      - True: the (x,y) coordinates of objects are expected to come from a simple two-column list.
          - Note: the (x,y) pairs should be in image (pixel) coordinates, with one pair per line in a .txt file.
      
      - False: set to False if not using a simple two-column list.   

In [ ]:
read_positions = True

lock_positions = False

positions_file = 'your_source_extractor_catalog.cat'

use_peaks = False

use_source_catalog_xy_coords = True # Read in Source Extractor catalog and select stars.

use_list_of_manual_xy_coords = False # Read in a two-column list of approximate (x,y) star coordinates.

if (use_source_catalog_xy_coords == True and use_list_of_manual_xy_coords == True):
    
    print('\033[1mWARNING: use_source_catalog_xy_coords and use_list_of_manual_xy_coords are both set to True!\033[0m')
    print('The code can read in either a Source Extractor type catalog, or a two-column list of coordinates.')
    print('Please set one of these options to false and run the above cell again to propagate your selection.')

## Setup Options: Source Extractor  <a class="anchor" id="setup5"><a>
[Table of Contents](#tag0)
    
***

**The user is REQUIRED to specify the following ONLY if *psf_type = 'stellar'* AND *use_source_catalog_xy_coords = True*:**
 - <font color=blue>*source_extractor_catalog*</font> - the filename of your Source Extractor catalog (string, .cat format).
      - MUST include columns for: ID (col 0), X (col 1), Y (col 2), class_star, mag_iso, and elongation.
      - Note: the mag_iso parameter may be any Source Extractor magnitude measure.
      

 - <font color=blue>*header_rows*</font> - the number of header rows contained in your catalog.
      - Note: Python starts indices at zero so e.g. 52 rows is entered as 51.
      
      
 - <font color=blue>*class_star_column*</font> - the catalog column number of the *class_star* parameter.   
      - Note: Python starts indices at zero so e.g. column 1 is entered as 0.
      
      
 - <font color=blue>*mag_iso_column*</font> - the catalog column number of the *mag_iso_column* parameter.     
      - Note: Python starts indices at zero so e.g. column 2 is entered as 1.
      
      
 - <font color=blue>*elongation_column*</font> - the catalog column number of the *elongation_column* parameter.    
      - Note: Python starts indices at zero so e.g. column 3 is entered as 2.     
     

 - <font color=blue>*class_star_limit*</font> - threshold for a star to be included in the model.
      - Range is 0 to 1 and recommend a setting of ~ 0.8 or greater.
      - Setting this too low will select non-stellar objects and too high results in too few stars. 
      
 
 - <font color=blue>*elongation_limit*</font> - threshold for a star to be included in the model.
      - Range is > 1 and recommend a setting of ~ 1.05 or greater (lower selects too few stars).
      - Setting this too low will result in too few stars and too high results in selecting non-stellar objects.    
      
      
 - <font color=blue>*manual_exclude*</font> - True or False.
      - True: Stars that are included in the manual exclusion lists below will be dropped.
          - In general, run the stellar selection code cell once, identify low quality stars visually, and add them to the exclusion list for the next run.
      - False: All stars automatically selected by the code will be used in the PSF model.  

In [ ]:
source_extractor_catalog = positions_file

header_rows = 51

class_star_column = 25

mag_iso_column = 26

elongation_column = 54

class_star_limit = 0.84

elongation_limit = 1.07

manual_exclude = True

## Setup Options: Manual Selection  <a class="anchor" id="setup6"><a>
[Table of Contents](#tag0)
    
***

**The user is REQUIRED to specify the following ONLY if *psf_type = 'stellar'* AND *manual_exclude = True*:**

The following cell allows the user to manually set the minimum distance for neighboring objects (*dists_from_star_limit*) and iteratively select out poor-quality stars from those that are automatically selected by the code. Stars are excluded by specifying a list of Source Extractor IDs, as well as a minimum magnitude limit for faint stars.

To guarantee that there are no contaminating objects, neighbors at distances of *dists_from_star_limit* < sqrt(2)\*rpix should be excluded (or larger when the field contains many extended objects. However, this may exlcude too many stars so using the half-width of the PSF size (*rpix*) and then manually excluding contaminated objects may be more practical.

In small and/or crowded fields, the *dists_from_star_limit* may be set to zero, as long as there are sufficient stars in the stack such that individual contaminating objects disappear in the median stack. There is no exact value for the minimum number of stars needed to "median out" contaminants, but > 5 may be considered a practical minimum.

In [ ]:
if manual_exclude == True:

    if radius_switch == 'inner':
        
        dists_from_star_limit = numpy.int(1.0*rpix)
        
        if my_filter == 'f336w':
            drop_stars = []
            mag_iso_limit = 30.0      
        
        if my_filter == 'f125w':
            drop_stars = []
            mag_iso_limit = 30.0
        
        if my_filter == 'f140w':
            drop_stars = []
            mag_iso_limit = 30.0

    if radius_switch == 'outer':
        
        dists_from_star_limit = numpy.int(1.0*rpix)
        
        if my_filter == 'f336w':
            drop_stars = []
            mag_iso_limit = 22.20        
    
        if my_filter == 'f125w':
            drop_stars = []
            mag_iso_limit = 22.20
    
        if my_filter == 'f140w':
            drop_stars = []
            mag_iso_limit = 22.20
    
if manual_exclude == False:
    
    drop_stars = [-1]

    mag_iso_limit = 98.0

***

## <font color=Green>Start of Main Code</font>
##### No user inputs are required below this cell. <a class="anchor" id="start"><a>
    
***
    
## Create folders for the results
[Table of Contents](#tag0)

In [ ]:
# Set directories and build folder tree.

os.chdir(main_directory)

try:
    os.mkdir(my_filter)
except:
    print('Directory already exists...')
    
code_directory = main_directory+my_filter+'/'


os.chdir(code_directory)

try:
    os.mkdir('results')
except:
    print('Directory already exists...')
    
    
os.chdir(code_directory+'/results/')

try:
    os.mkdir(export_directory)
except:
    print('Directory already exists...')
    
results_directory = code_directory+'results/'+export_directory+'/'


os.chdir(results_directory)

try:
    os.mkdir(psf_type)
except:
    print('Directory already exists...')


if dual_psf == True:
    
    dir1 = 'inner'
    dir2 = 'outer'
    
    os.chdir(results_directory+psf_type+'/')
    
    try:
        os.mkdir(dir1)
    except:
        print('Directory already exists...')
    try:
        os.mkdir(dir2)
    except:
        print('Directory already exists...')


if (dual_psf == True and radius_switch == 'inner'):
    
    dir_out = dir1+'/'

if (dual_psf == True and radius_switch == 'outer'):
    
    dir_out = dir2+'/'
    
if dual_psf == False:
    
    dir1 = ''
    dir2 = ''

    
print('\nThe code_directory is:', code_directory)

print('\nThe results_directory is:', results_directory)

os.chdir(code_directory)

In [ ]:
# Determine the coordinates for the center of the image.
# This value may be set manually to offset the center and/or if the file header is incorrect.

os.chdir(main_directory+my_filter)

hdul = fits.open(science_drizzle)

xcenter = (((hdul[0].header['NAXIS1']) / 2.0) + 1.0)

ycenter = (((hdul[0].header['NAXIS2']) / 2.0) + 1.0)

hdul.close()

print('\nThe calculated image center is located at (x,y) = ('+str(xcenter)+', '+str(ycenter)+').')
print('\nIf this is not the desired center then set the values manually in this cell.')

#xcenter = 

#ycenter = 

## Select stars from the users Source Extractor catalog <a class="anchor" id="select"><a>
[Table of Contents](#tag0)

In [ ]:
if (psf_type == 'stellar' and use_source_catalog_xy_coords == True):
    
    # Import the Source Extractor catalog.
    catalog = numpy.loadtxt(fname=source_extractor_catalog, skiprows = header_rows)

    # Extract the IDs and (x, y) coordinates from the 1st, 2nd and 3rd columns.
    ids = catalog[:,0].astype(int)
    xs = catalog[:,1]
    ys = catalog[:,2]

    # Extract class_star, mag_iso, and elongation for each object.
    class_star = catalog[:,class_star_column]
    mag_iso = catalog[:,mag_iso_column]
    elongation = catalog[:,elongation_column]

    # Calculate the distance of each source from the center of the field.
    dists = numpy.sqrt((xs-xcenter)**2.0 + (ys-ycenter)**2.0)

    print('There are', len(ids), 'entries in the catalog. \n')
    print(class_star)
    print('The min and max class_star values are:', min(class_star), ',', max(class_star), '\n')
    print(mag_iso)
    print('The min and max mag_iso values are:', min(mag_iso), ',', max(mag_iso), '\n')
    print(elongation)
    print('The min and max elongation values are:', min(elongation), ',', max(elongation), '\n')
    print(numpy.around(dists, decimals=3))
    print('The min and max dists values are: ~', int(min(dists)), 'and ~', int(max(dists)), 'pixels. \n')

    # Calculate the number of objects in each region.
    print('The number of objects in the deep region is:', sum(i < rmax for i in dists))
    print('The number of objects in the shal region is:', sum(i >= rmax for i in dists))
    print('The total number of deep + shall objects is:', 
          sum(i < rmax for i in dists)+sum(dum >= rmax for dum in dists))

## Identify isolated stars based on neighbor, elongation, and magnitude limits <a class="anchor" id="clean"><a>
[Table of Contents](#tag0)

In [ ]:
if (psf_type == 'stellar' and use_source_catalog_xy_coords == True):
    
    # Search through all objects to locate isolated stars based on their class_star and distance from other objects.
    stars_list = []

    # Set search criteria to deep 'inner' or shallow 'outer' region.
    if radius_switch == 'inner':
        print('Searching for stars at < rmax (inner)...')

    if radius_switch == 'outer':
        print('Searching for stars at > rmax (outer)...')       

    # Search over all objects in the catalog.
    for j in range(len(ids)):
    
        # Set search criteria to deep 'inner' or shallow 'outer' region.
        if radius_switch == 'inner':
            gt_lt = operator.lt(dists[j], rmax)

        if radius_switch == 'outer':
            gt_lt = operator.gt(dists[j], rmax)
    
        if class_star[j] >= class_star_limit and gt_lt and elongation[j] <= elongation_limit\
        and ids[j] not in drop_stars and mag_iso[j] < mag_iso_limit:
        
            # Calculate the distance to all objects and determine if the star is isolated.
            dists_from_star = numpy.sqrt((xs-xs[j])**2.0 + (ys-ys[j])**2.0)
        
            # Determine the closest neighbor, excluding the object itself.
            closest_neighbor = min(dists_from_star[numpy.nonzero(dists_from_star)])
        
            # If the star is isolated then append it to the list.
            if closest_neighbor >= dists_from_star_limit:
            
                stars_list.append(catalog[j])            
                print('Star for PSF model: ID', ids[j], 
                      '- class_star =', class_star[j], 
                      '- closest =', int(closest_neighbor), 'pixels', 
                      '- elongation =', elongation[j], 
                      '- mag_iso =', mag_iso[j]) 

    # Print the IDs of the identified stars.
    star_ids = [k[0] for k in stars_list]
    print('\nThe total number of stars is:', len(stars_list))

## Import the science drizzle for stellar extraction <a class="anchor" id="import"><a>
[Table of Contents](#tag0)

In [ ]:
if (psf_type == 'stellar' and use_source_catalog_xy_coords == True):
    
    # Import the image mosaic for stellar extraction.
    mosaic = fits.open(science_drizzle)
    image = mosaic[0].data
    mosaic.info()

    # Extract the x and y coordinates of the stars.
    xis = numpy.array([k[1] for k in stars_list])
    yis = numpy.array([k[2] for k in stars_list])

    print('\n xis =', xis)
    print('\n yis =', yis)

    # Determine the min, max, and mean offset error from rounding to integer pixel centers.
    xoffsets = (xis - numpy.rint(xis))
    xoffmean = numpy.around((numpy.mean(numpy.absolute(xoffsets))), decimals=3)
    xoffsdev = numpy.around(numpy.std(numpy.absolute(xoffsets)), decimals=3)
    xoffmin = numpy.around(numpy.amin(numpy.absolute(xoffsets)), decimals=3)
    xoffmax = numpy.around(numpy.amax(numpy.absolute(xoffsets)), decimals=3)

    yoffsets = (yis - numpy.rint(yis))
    yoffmean = numpy.around((numpy.mean(numpy.absolute(yoffsets))), decimals=3)
    yoffsdev = numpy.around(numpy.std(numpy.absolute(yoffsets)), decimals=3)
    yoffmin = numpy.around(numpy.amin(numpy.absolute(yoffsets)), decimals=3)
    yoffmax = numpy.around(numpy.amax(numpy.absolute(yoffsets)), decimals=3)

## Import the empirical drizzle for model extraction <a class="anchor" id="import2"><a>
[Table of Contents](#tag0)

In [ ]:
if psf_type == 'empirical':
    
    os.chdir(code_directory)
    
    # Set the PSF model drizzle filename fixed by make_model_star_image().
    psf_drizzle = science_drizzle.split('_sci', 1)[0]+'.fits'

    # Import the image mosaic for extraction.
    mosaic = fits.open(psf_drizzle)
    image = mosaic[1].data
    mosaic.info()    
    
    # Initialize arrays.
    xis_all = []
    yis_all = []
    xis = []
    yis = []
    psf_ids_all = []    
    psf_ids = []
    
    
    if use_list_of_manual_xy_coords == True:
        
        print('Reading PSF locations from', positions_file+'...\n')
    
        my_input_coordinates = numpy.loadtxt(positions_file)

        xis_all = my_input_coordinates[:,0]

        yis_all = my_input_coordinates[:,1]

        psf_ids_all = numpy.arange(0, len(my_input_coordinates), 1)  
    
    
    if read_positions == False:
    
        # See: https://photutils.readthedocs.io/en/stable/detection.html
        # photutils.detection.find_peaks(data, threshold, box_size=3, footprint=None, mask=None, 
        # border_width=None, npeaks=inf, centroid_func=None, subpixel=False, error=None, wcs=None)
    
        print('\nSearching for PSFs in the drizzle...\n')
    
        peaks_table = find_peaks(image, threshold=0.01, box_size=10.0)
    
        peaks_table['peak_value'].info.format = '%.6g'
    
        print('peak_finder has located', len(peaks_table), 'PSFs.\n')
    
        print(peaks_table, '\n')
    
        xis_all = peaks_table['x_peak']
    
        yis_all = peaks_table['y_peak']
    
        psf_ids_all = numpy.arange(0, len(peaks_table), 1)

        
    # Search through all PSFs to locate those based on rmax criteria.
    if dual_psf == True:
        
        # Calculate the distance of each PSF from the center of the field.
        dists = numpy.sqrt((xis_all-xcenter)**2.0 + (yis_all-ycenter)**2.0)

        # Set search criteria to deep 'inner' or shallow 'outer' region.
        if radius_switch == 'inner':
            print('Searching for PSFs at < rmax (inner)...')

        if radius_switch == 'outer':
            print('Searching for PSFs at > rmax (outer)...')       

        # Search over all PSFs in the image.
        for j in range(len(psf_ids_all)):
    
            # Set search criteria to deep 'inner' or shallow 'outer' region.
            if radius_switch == 'inner':
                gt_lt = operator.lt(dists[j], rmax)

            if radius_switch == 'outer':
                gt_lt = operator.gt(dists[j], rmax)
    
            # If the object is inside the circle then append its location.
            if gt_lt:
                xis.append(xis_all[j])
                yis.append(yis_all[j])
                psf_ids.append(psf_ids_all[j])

    # If modeling the entire image then keep all objects.
    if dual_psf == False:
        
        psf_ids = psf_ids_all
        xis = xis_all
        yis = yis_all

    # Print the IDs of the identified PSFs.
    print('\nThe PSF IDs are:', psf_ids)
    print('\nThe total number of PSFs is:', len(psf_ids))
    
    # Pass the PSF IDs to the star_ids variable for compatibility with all functions.
    star_ids = psf_ids  

## Extract subimages of each object that will be stacked <a class="anchor" id="extract"><a>
[Table of Contents](#tag0)

In [ ]:
# Extract a subimage centered on the star.
image_array = []

for i in range(len(xis)):
    
    # Convert to integer and round to nearest value.
    xi = int(numpy.rint(xis[i]))
    yi = int(numpy.rint(yis[i]))
    star_id = star_ids[i]
    
    # Print the x, y coordinates and extract each star image.
    print('Star ID ' + str(int(star_id)) + ':'  + ' (x,y) = (' + str(xi) + ', ' + str(yi) + ')')
    
    # Python switches the "slow" axis so reverse x and y.
    if subimg_shape == 'even':
        subimage = image[yi-rpix:yi+rpix, xi-rpix:xi+rpix]
    if subimg_shape == 'odd':
        subimage = image[yi-rpix:yi+rpix+1, xi-rpix:xi+rpix+1]
    
    # Determine the location of the maximum flux.
    peak_location = numpy.unravel_index(numpy.argmax(subimage, axis=None), subimage.shape)
    print('The subimage peak flux (x,y) = (' + str(peak_location[1]) + ', ' + str(peak_location[0]) + ')')
    
    # Scale to maximum flux so all stars peak at unity.
    if scale_stars == True:
        print('Scaling the stars peak flux to unity...')
        subimage = (subimage/numpy.amax(subimage))
        
    # Report extractions from peak_finder that do not contain a star.
    if (peak_location[1] == 0 and peak_location[0] == 0):
        
        print('This object is outside the data region and will be excluded.\n')
    
    # Protect against peak_finder results that don't have a star.
    #if peak_location[1] != 0 and peak_location[0] != 0:
    # Protect against peak_finder results that don't have a star and are near the edge.    
    if (peak_location[1] != 0 and peak_location[0] != 0\
    and peak_location[1] > rpix/5.0 and peak_location[0] > rpix/5.0\
    and peak_location[1] < 1.8*rpix and peak_location[0] < 1.8*rpix):
    
        # Append subimage to image_array.
        image_array.append(subimage)
    
        # Plot the resulting subimages.
        figure, mysubplot = matplotlib.pyplot.subplots(1, 3, figsize=(11, 11))
        mysubplot[0].imshow(subimage, cmap='gray', vmin=0.0, vmax=numpy.amax(subimage), origin='lower')
        mysubplot[1].imshow(subimage, cmap='gray', vmin=0.0, vmax=numpy.amax(subimage)/10.0, origin='lower')
        mysubplot[2].imshow(subimage, cmap='gray', vmin=0.0, vmax=numpy.amax(subimage)/100.0, origin='lower')
        matplotlib.pyplot.tight_layout()
        print('\n            100% MAX:                       10% MAX:                       1% MAX:') 
        matplotlib.pyplot.show()
        
    else:
        print('The peak flux is too close to the edge of the subimage.')        

## Define a recursive mean and median function <a class="anchor" id="meanfunction"><a>
[Table of Contents](#tag0)

In [ ]:
# Define a mean and median function that can be called repetitively.
# e.g. image_combine(input_array = image_array, export_file = export_name)

def image_combine(input_array, export_file):
    print('Creating a mean and median image stack...')

    # Mean of star images.
    mean_image = numpy.mean(input_array, axis=0)

    # Median of star images.
    median_image = numpy.median(input_array, axis=0)

    # Scale the final images to have a total flux of unity.
    if scale_image == True:
        print('Scaling the stacks peak flux to unity...')
        mean_image = (mean_image/numpy.sum(mean_image))
        median_image = (median_image/numpy.sum(median_image))
        extension = '_normalized.fits'

    if scale_image == False:
        extension = '.fits'
    
    mean_flux = numpy.sum(mean_image)
    median_flux = numpy.sum(median_image)
    
    print('The mean_image total flux =', numpy.around(mean_flux, decimals=5))
    print('The median_image total flux =', numpy.around(median_flux, decimals=5))
    
    # Plot the resulting subimages.
    figure, mysubplot = matplotlib.pyplot.subplots(1, 3, figsize=(11, 11), sharex=True, sharey=True)
    mysubplot[0].imshow(mean_image, cmap='gray', vmin=0.0, vmax=numpy.amax(mean_image), origin='lower')
    mysubplot[1].imshow(mean_image, cmap='gray', vmin=0.0, vmax=numpy.amax(mean_image)/10.0, origin='lower')
    mysubplot[2].imshow(mean_image, cmap='gray', vmin=0.0, vmax=numpy.amax(mean_image)/100.0, origin='lower')
    matplotlib.pyplot.tight_layout()
    print('\nMean:        100% MAX:                      10% MAX:                       1% MAX:') 
    matplotlib.pyplot.show()

    figure, mysubplot = matplotlib.pyplot.subplots(1, 3, figsize=(11, 11), sharex=True, sharey=True)
    mysubplot[0].imshow(median_image, cmap='gray', vmin=0.0, vmax=numpy.amax(median_image), origin='lower')
    mysubplot[1].imshow(median_image, cmap='gray', vmin=0.0, vmax=numpy.amax(median_image)/10.0, origin='lower')
    mysubplot[2].imshow(median_image, cmap='gray', vmin=0.0, vmax=numpy.amax(median_image)/100.0, origin='lower')
    matplotlib.pyplot.tight_layout()
    print('\nMedian:      100% MAX:                      10% MAX:                       1% MAX:') 
    matplotlib.pyplot.show()
    
    if (export_results == True and dual_psf == False):
        my_psf = fits.PrimaryHDU(median_image)
        my_psf.writeto(results_directory+psf_type+'/'+export_file+'_median'+extension, overwrite=True)
        my_psf = fits.PrimaryHDU(mean_image)
        my_psf.writeto(results_directory+psf_type+'/'+export_file+'_mean'+extension, overwrite=True)          
        
    if (export_results == True and dual_psf == True):
        my_psf = fits.PrimaryHDU(median_image)
        my_psf.writeto(results_directory+psf_type+'/'+dir_out+export_file+'_median'+extension, overwrite=True)
        my_psf = fits.PrimaryHDU(mean_image)
        my_psf.writeto(results_directory+psf_type+'/'+dir_out+export_file+'_mean'+extension, overwrite=True)
    
    # Return the mean and median images.
    return mean_image, median_image
    os.chdir(code_directory)
    
# End of function.

## Define a recursive profile fitting function <a class="anchor" id="fitfunction"><a>
[Table of Contents](#tag0)

In [ ]:
# Fit the data using either Gaussian2D or Moffat2D functions on the interpolated images.
# e.g. fit_profile(model_type = 'gaussian/moffat', input_data = image_array, fit_info = True/False)

def fit_profile(model_type, input_data, fit_info):

    x_centroids = []
    y_centroids = []

    for i in range(len(input_data)):
    
        # Print the x, y coordinates and extract star image.
        print('Star ID ' + str(int(star_ids[i])) + ':')
    
        # The data are the interpolated images.
        data = input_data[i]
    
        # Create a finer grid of (x, y) coordinates for interpolation.
        x_z = numpy.around(numpy.linspace(1, (2*rpix), num=2*interp_grid_steps*rpix, endpoint=True), decimals=5)
        y_z = x_z
    
        # Declare the fitting algorithm.
        fit_method = fitting.LevMarLSQFitter()
        amplitude = numpy.max(data)
    
        # Declare the type of model to fit.
        if model_type == 'gaussian':
            fit_model = models.Gaussian2D(amplitude, x_mean=interp_grid_steps*rpix, y_mean=interp_grid_steps*rpix)
        if model_type == 'moffat':
            fit_model = models.Moffat2D(amplitude, x_0=interp_grid_steps*rpix, y_0=interp_grid_steps*rpix)
        
        # Generate the model array.            
        ximg, yimg = numpy.indices(data.shape)
    
        # Fit the model to the data and evaluate.
        fit_result = fit_method(fit_model, ximg, yimg, data, maxiter=max_iterations)
        model = fit_result(ximg, yimg)
        residual = (data - model) 
    
        # Find the locations of the peak flux value in the data and model.
        peak_location_data = numpy.unravel_index(numpy.argmax(data, axis=None), data.shape)
        peak_location_model = numpy.unravel_index(numpy.argmax(model, axis=None), model.shape)
    
        peak_location_data_x = x_z[peak_location_data[1]]
        peak_location_data_y = y_z[peak_location_data[0]]
        peak_location_model_x = x_z[peak_location_model[1]]
        peak_location_model_y = y_z[peak_location_model[0]]
    
        print('Peak Data Flux Indices  (y,x):', peak_location_data)
        print('Peak Data Flux Center   (y,x):', peak_location_data_y, peak_location_data_x)
        print('Peak Model Flux Indices (y,x):', peak_location_model)
        print('Peak Model Flux Center  (y,x):', peak_location_model_y, peak_location_model_x)
    
        # Plot the data, model, and residuals side-by-side.
        figure, mysubplot = matplotlib.pyplot.subplots(1, 3, figsize=(11, 11))
        mysubplot[0].imshow(data, cmap='gray', vmin=0.0, vmax=numpy.amax(data)/100.0, origin='lower')
        mysubplot[1].imshow(model, cmap='gray', vmin=0.0, vmax=numpy.amax(model)/100.0, origin='lower')
        mysubplot[2].imshow(residual, cmap='gray', vmin=0.0, vmax=numpy.amax(residual)/100.0, origin='lower')
        matplotlib.pyplot.tight_layout()
        print('\n              DATA:                          MODEL:                           RESIDUAL:') 
        matplotlib.pyplot.show()
    
        # Extract the model (x, y) centers.    
        x_centroids.append(peak_location_model_x)    
        y_centroids.append(peak_location_model_y)
    
        # Print detailed information of the fit if there is an error.
        if fit_info == True:
            print('fit_method.fit_info = ', fit_method.fit_info)
    
    # Announce completion.
    if speak == True:
        os.system("say 'fitting complete'")
        
    # Return the lists of (x,y) centroids.
    return x_centroids, y_centroids
            
# End of function.

## Interpolate each extracted subimage <a class="anchor" id="interpolate"><a>
[Table of Contents](#tag0)

In [ ]:
# The grid to interpolate is the same size as the original image.
image_array_interpolated = []

use_peaks_interp_x = []
use_peaks_interp_y = []

for i in range(len(image_array)):
  
    # Convert to integer and round to nearest value.
    xi = int(numpy.rint(xis[i]))
    yi = int(numpy.rint(yis[i]))
    star_id = star_ids[i]
    
    # Print the x, y coordinates and extract each star image.
    print('Star ID ' + str(int(star_id)) + ':'  + ' (x,y) = (' + str(xi) + ', ' + str(yi) + ')')
    
    # Python switches the "slow" axis so reverse x and y.
    if subimg_shape == 'even':
        
        ximg = list(range(0, 2*rpix))
        yimg = ximg
        zimg = image[yi-rpix:yi+rpix, xi-rpix:xi+rpix]
        
        # Create a finer grid of (x, y) coordinates for interpolation.
        x_z = numpy.around(numpy.linspace(1, (2*rpix), num=2*interp_grid_steps*rpix, endpoint=True), decimals=5)
        y_z = x_z
        
    if subimg_shape == 'odd':
            
        ximg = list(range(0, (2*rpix)+1))
        yimg = ximg
        zimg = image[yi-rpix:yi+rpix+1, xi-rpix:xi+rpix+1]
        
        # Create a finer grid of (x, y) coordinates for interpolation.
        x_z = numpy.around(numpy.linspace(1, ((2*rpix)+1), num=2*interp_grid_steps*rpix, endpoint=True), decimals=5)
        y_z = x_z
    
    # Determine the location of the maximum flux.
    peak_location = numpy.unravel_index(numpy.argmax(zimg, axis=None), zimg.shape)
    print('The subimage peak flux (x,y) = (' + str(peak_location[1]) + ', ' + str(peak_location[0]) + ')')
        
    # Scale to maximum flux so all stars peak at unity.
    if scale_stars == True:
        print('Scaling the stars peak flux to unity...')
        zimg = (zimg/numpy.amax(zimg))
     
    # Report extractions from peak_finder that do not contain a star.
    if (peak_location[1] == 0 and peak_location[0] == 0):
        
        print('This object is outside the data region and will be excluded.\n')
    
    # Protect against peak_finder results that don't have a star.
    #if peak_location[1] != 0 and peak_location[0] != 0:
    # Protect against peak_finder results that don't have a star and are near the edge.    
    if (peak_location[1] != 0 and peak_location[0] != 0\
    and peak_location[1] > rpix/5.0 and peak_location[0] > rpix/5.0\
    and peak_location[1] < 1.8*rpix and peak_location[0] < 1.8*rpix):        
    
        # Interpolate to a finer grid.
        f_z = scipy.interpolate.interp2d(ximg, yimg, zimg, kind='cubic')
        
        # Evaluate the interpolation function.
        interpolated_image = f_z(x_z, y_z)
    
        # Append subimage to image_array.
        image_array_interpolated.append(interpolated_image)
    
        # Save the interpolated peak (x,y) locations for use_peaks = True. #M.D.R 12/21/2020
        if use_peaks == True:
            peak_interp = numpy.unravel_index(numpy.argmax(interpolated_image, axis=None), interpolated_image.shape)
            use_peaks_interp_x.append((peak_interp[1]/interpolated_image.shape[1])*(2*rpix)+0.5)
            use_peaks_interp_y.append((peak_interp[0]/interpolated_image.shape[0])*(2*rpix)+0.5)        
        
        # Plot the data, model, and residuals side-by-side.
        figure, mysubplot = matplotlib.pyplot.subplots(1, 2, figsize=(8, 8))
        mysubplot[0].imshow(zimg, cmap='gray', vmin=0.0, vmax=numpy.amax(zimg)/100.0, origin='lower')
        mysubplot[1].imshow(interpolated_image, cmap='gray', vmin=0.0, vmax=numpy.amax(interpolated_image)/100.0, origin='lower')
        matplotlib.pyplot.tight_layout()
        print('\n              Original:                        Interpolated:') 
        matplotlib.pyplot.show()
    
# Announce completion.
if speak == True:
    os.system("say 'interpolation complete'")

## Call the fitting function on the interpolated images <a class="anchor" id="fitting"><a>
[Table of Contents](#tag0)

In [ ]:
if (parallel_mode == False and lock_positions == False and use_peaks == False):

    # Call the fitting function.
    my_centroids = fit_profile(model_type = fit_type, input_data = image_array_interpolated, fit_info = print_fit)

In [ ]:
if (parallel_mode == False and lock_positions == False and use_peaks == False):
    print(my_centroids)

In [ ]:
if (parallel_mode == True and lock_positions == False and use_peaks == False):

    # fit_profile_parallel.py
    import multiprocessing
    
    from multiprocessing import Pool
    
    from fit_profile_parallel import fit_profile_parallel
    
    if number_of_cores == 'max':
        
        number_of_cores = multiprocessing.cpu_count()
        
    print('Initializing parallel mode using', number_of_cores, 'cores.\n')
            
    pool = Pool(number_of_cores)    

    print('Beginning parallelized model fitting...\n')
    
    print('There are', len(image_array_interpolated), 'profiles to fit.\n')

    input_data = image_array_interpolated
    
    pool_list = []
    
    j=0
    
    for i in input_data:
        pool_list.append(('moffat', input_data[j], False, star_ids[j], rpix, interp_grid_steps, max_iterations, speak))
        j=j+1
        
    print('Created a parallel list for', len(pool_list), 'objects.\n')

    my_centroids_parallel = pool.starmap(fit_profile_parallel, pool_list)

    pool.close()

In [ ]:
if (parallel_mode == True and lock_positions == False and use_peaks == False):
    print(my_centroids_parallel)

In [ ]:
if (parallel_mode == True and lock_positions == False and use_peaks == False):

    # Separate the concatenated (x,y) centroid lists produced by parallel_mode.
    x_sep = [i[0] for i in my_centroids_parallel]
    y_sep = [i[1] for i in my_centroids_parallel]

    x_sep = list(chain.from_iterable(x_sep))
    y_sep = list(chain.from_iterable(y_sep))

    my_centroids_parallel_formatted = (x_sep, y_sep)
                                  
    print(my_centroids_parallel_formatted)
    
    # Test to ensure non-parallel and parallel runs produce identical ordered results.
    #my_centroids == my_centroids_parallel_formatted

    my_centroids = my_centroids_parallel_formatted

## Calculate the image shifts from the model fits <a class="anchor" id="shifts"><a>
[Table of Contents](#tag0)

In [ ]:
if (lock_positions == False and use_peaks == False):

    # Determine the image shifts based on the model centers and the desired image center.
    x_centroids = my_centroids[0]
    y_centroids = my_centroids[1]

    print('\nx_centroids =', x_centroids)
    print('\ny_centroids =', y_centroids)

    # Create a list filled with the desired center value.
    my_center = []

    for i in range(len(x_centroids)):
        my_center.append(new_center)
    print('\nmy_center =', my_center)

    # Calculate the (x, y) offsets.
    xshifts = numpy.subtract(my_center, x_centroids)
    yshifts = numpy.subtract(my_center, y_centroids)

    print('\nxshifts =', xshifts)
    print('\nyshifts =', yshifts)

In [ ]:
if lock_positions == True:
    
    # Determine the image shifts based on the model centers and the desired image center.
    x_centroids = xis
    y_centroids = yis

# Use the interpolated peak (x,y) locations if use_peaks = True.
if use_peaks == True:

    x_centroids = use_peaks_interp_x
    y_centroids = use_peaks_interp_y

    print('\nx_centroids =', x_centroids)
    print('\ny_centroids =', y_centroids)

    # Create a list filled with the desired center value.
    my_center = []

    for i in range(len(x_centroids)):
        my_center.append(new_center)
        if i==0: print('\nmy_center = {}'.format(my_center))

    # Calculate the (x, y) offsets.
    xshifts = numpy.subtract(my_center, x_centroids)
    yshifts = numpy.subtract(my_center, y_centroids)

    # Make the offsets relative to the postage-stamps instead of the image.
    # if use_peaks == False: # Confirm these shifts are correct in all cases.

    #     xshifts = (xshifts + numpy.rint(x_centroids)) - new_center + 1.0
    #     yshifts = (yshifts + numpy.rint(y_centroids)) - new_center + 1.0

    print('\nxshifts =', xshifts)
    print('\nyshifts =', yshifts)

## Align the interpolated images and calculate the median <a class="anchor" id="align"><a>
[Table of Contents](#tag0)

In [ ]:
# Determine shift recalling x and y are reversed in python manipulation.
# See https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.shift.html for 'mode' choices.
image_array_shifted = []

# Zoom in factor for below plots.
imgzoom = (1./interp_grid_steps)*6.0

for i in range(len(xshifts)):
    
    #xshift > Positive = shift right. Negative = shift left.
    #yshift > Positive = shift up. Negative = shift down.
    
    # Model fit values.
    xshift = numpy.around(xshifts[i]*interp_grid_steps, decimals=5)
    yshift = numpy.around(yshifts[i]*interp_grid_steps, decimals=5)
    print('The Offsets (x, y) =', xshift, yshift)
    
    # Shift images by xshift and yshift.
    shifted_image = scipy.ndimage.shift(image_array_interpolated[i], [yshift, xshift], mode='mirror')
    
    # Set approximate zoomed in center and subtract 0.5 for index axes.
    xylimit = ((new_center-0.5)*interp_grid_steps)
    
    # Set imshow range to +/- some % of total.
    xyrange = (xylimit/(imgzoom*interp_grid_steps))
    
    peak_original = numpy.unravel_index(numpy.argmax(image_array_interpolated[i], axis=None), image_array_interpolated[i].shape)
    peak_original_x = peak_original[1]
    peak_original_y = peak_original[0]
    print('Before Shift: Peak Model Flux Center (x,y):', peak_original_x, peak_original_y)
    
    peak_shifted = numpy.unravel_index(numpy.argmax(shifted_image, axis=None), shifted_image.shape)
    peak_shifted_x = peak_shifted[1]
    peak_shifted_y = peak_shifted[0]
    print('After Shift : Peak Model Flux Center (x,y):', peak_shifted_x, peak_shifted_y, '\n')
    
    # Append subimage to image_array.
    image_array_shifted.append(shifted_image)
    
    # Plot the data and shifted data side-by-side.
    figure, mysubplot = matplotlib.pyplot.subplots(1, 2, figsize=(10, 10), sharex=True, sharey=True)
    matplotlib.pyplot.xlim(xylimit-xyrange, xylimit+xyrange)
    matplotlib.pyplot.ylim(xylimit-xyrange, xylimit+xyrange)

    mysubplot[0].imshow(image_array_interpolated[i], cmap='gray', vmin=0.0,\
                        vmax=numpy.amax(image_array_interpolated[i]), origin='lower')
    mysubplot[0].scatter(xylimit, xylimit, s=1000000, c='green', marker='+')
    mysubplot[0].scatter(peak_original_x, peak_original_y, s=300, c='black', marker='x')

    mysubplot[1].imshow(shifted_image, cmap='gray', vmin=0.0,\
                        vmax=numpy.amax(shifted_image), origin='lower')    
    mysubplot[1].scatter(xylimit, xylimit, s=1000000, c='green', marker='+')
    mysubplot[1].scatter(peak_shifted_x, peak_shifted_y, s=300, c='black', marker='x')

    matplotlib.pyplot.tight_layout()
    print('                   Original:                                 Shifted:') 
    matplotlib.pyplot.show()
    
# The below images show the star before and after re-centering, where the "x" denotes the peak flux location.
# The "x" will not be perfectly centered unless use_peaks = True, as the data and model peaks are slightly offset.

## Calculate the mean and median image stack <a class="anchor" id="combine"><a>
[Table of Contents](#tag0)

In [ ]:
# Call the mean and median function. Save the output arrays for reinterpolation to the native pixel scale.
mean_out, median_out = image_combine(input_array = image_array_shifted, 
                                     export_file = export_name+'_interpolated_'+fit_type)

## Reinterpolate the stack to the native pixel scale <a class="anchor" id="reinterpolate"><a>
[Table of Contents](#tag0)

In [ ]:
# Reinterpolate the median and mean images to their original image scales.

# Mean.
zimg = mean_out
# Evaluate the interpolation function to the original grid.
f_z = scipy.interpolate.interp2d(x_z, y_z, zimg, kind='cubic')
mean_reinterpolation = f_z(ximg, yimg)

# Median.
zimg = median_out
# Evaluate the interpolation function to the original grid.
f_z = scipy.interpolate.interp2d(x_z, y_z, zimg, kind='cubic')    
median_reinterpolation = f_z(ximg, yimg)

# Scale the final images to have a total flux of unity.
if scale_image == True:
    
    mean_reinterpolation = (mean_reinterpolation/numpy.sum(mean_reinterpolation))
    median_reinterpolation = (median_reinterpolation/numpy.sum(median_reinterpolation))

# Calculate the total flux of each image.
mean_flux_interp = numpy.sum(mean_reinterpolation)
median_flux_interp = numpy.sum(median_reinterpolation)
print('The mean_image total flux =', mean_flux_interp)
print('The median_image total flux =', median_flux_interp)

# Plot the resulting subimages.
figure, mysubplot = matplotlib.pyplot.subplots(1, 3, figsize=(11, 11), sharex=True, sharey=True)
mysubplot[0].imshow(mean_reinterpolation, cmap='gray', vmin=0.0, vmax=numpy.amax(mean_reinterpolation), origin='lower')
mysubplot[1].imshow(mean_reinterpolation, cmap='gray', vmin=0.0, vmax=numpy.amax(mean_reinterpolation)/10.0, origin='lower')
mysubplot[2].imshow(mean_reinterpolation, cmap='gray', vmin=0.0, vmax=numpy.amax(mean_reinterpolation)/100.0, origin='lower')
matplotlib.pyplot.tight_layout()
print('\nMean:        100% MAX:                      10% MAX:                       1% MAX:') 
matplotlib.pyplot.show()

# Plot the resulting subimages.
figure, mysubplot = matplotlib.pyplot.subplots(1, 3, figsize=(11, 11), sharex=True, sharey=True)
mysubplot[0].imshow(median_reinterpolation, cmap='gray', vmin=0.0, vmax=numpy.amax(median_reinterpolation), origin='lower')
mysubplot[1].imshow(median_reinterpolation, cmap='gray', vmin=0.0, vmax=numpy.amax(median_reinterpolation)/10.0, origin='lower')
mysubplot[2].imshow(median_reinterpolation, cmap='gray', vmin=0.0, vmax=numpy.amax(median_reinterpolation)/100.0, origin='lower')
matplotlib.pyplot.tight_layout()
print('\nMedian:      100% MAX:                      10% MAX:                       1% MAX:') 
matplotlib.pyplot.show()

# Set the directory for output files.
if (dual_psf == True and radius_switch == 'inner'):
    my_extension = dir1
    
if (dual_psf == True and radius_switch == 'outer'):
    my_extension = dir2
    
if dual_psf == False:
    my_extension = ''
    

# Export the images as DS9 readable .fits files.
if export_results == True:
    
    if dual_psf == True:
        my_psf = fits.PrimaryHDU(mean_reinterpolation)
        my_psf.writeto(results_directory+psf_type+'/'+dir_out+export_name+'_'+psf_type+'_mean_'+my_extension+'.fits', overwrite=True)
        my_psf = fits.PrimaryHDU(median_reinterpolation)
        my_psf.writeto(results_directory+psf_type+'/'+dir_out+export_name+'_'+psf_type+'_median_'+my_extension+'.fits', overwrite=True)

    elif dual_psf == False:
        my_psf = fits.PrimaryHDU(mean_reinterpolation)
        my_psf.writeto(results_directory+psf_type+'/'+export_name+'_'+psf_type+'_mean_'+my_extension+'.fits', overwrite=True)
        my_psf = fits.PrimaryHDU(median_reinterpolation)
        my_psf.writeto(results_directory+psf_type+'/'+export_name+'_'+psf_type+'_median_'+my_extension+'.fits', overwrite=True)    

os.chdir(code_directory)

# Announce completion.
if speak == True:
    os.system("say 'modeling complete'")

## Confirm that flux was conserved <a class="anchor" id="check"><a>
[Table of Contents](#tag0)

In [ ]:
# Confirm that flux is conserved.
mean_image = numpy.mean(image_array, axis=0)
median_image = numpy.median(image_array, axis=0)

# Scale the final images to have a total flux of unity.
if scale_image == True:
    mean_image = (mean_image/numpy.sum(mean_image))
    median_image = (median_image/numpy.sum(median_image))

mean_flux = numpy.sum(mean_image)
median_flux = numpy.sum(median_image)

# Compare to the fluxes of the interpolated images.
avg = (mean_flux + mean_flux_interp)/2.0
mean_percent_diff = (100.0*((mean_flux - mean_flux_interp)/avg))
print('The mean   image percent difference in flux before and after interpolation:', mean_percent_diff, '%')

avg = (median_flux + median_flux_interp)/2.0
median_percent_diff = (100.0*((median_flux - median_flux_interp)/avg))
print('The median image percent difference in flux before and after interpolation:', median_percent_diff, '%')

## Measure the PSF model width <a class="anchor" id="fwhm"><a>
[Table of Contents](#tag0)

In [ ]:
# Declare the fitting algorithm.
data = median_reinterpolation
fit_method = fitting.LevMarLSQFitter()
amplitude = numpy.max(data)
    
# Declare the type of model to fit.
fit_model = models.Gaussian2D(amplitude, x_mean=new_center, y_mean=new_center)
        
# Generate the model array.            
ximg, yimg = numpy.indices(data.shape)
    
# Fit the model to the data and evaluate.
fit_result = fit_method(fit_model, ximg, yimg, data, maxiter=max_iterations)
model = fit_result(ximg, yimg)
residual = (data - model) 
    
# Plot the data, model, and residuals side-by-side.
figure, mysubplot = matplotlib.pyplot.subplots(1, 3, figsize=(11, 11))
mysubplot[0].imshow(data, vmin=0.0, vmax=numpy.amax(data)/10.0, origin='lower')
mysubplot[1].imshow(model, vmin=0.0, vmax=numpy.amax(model)/10.0, origin='lower')
mysubplot[2].imshow(residual, vmin=0.0, vmax=numpy.amax(residual)/10.0, origin='lower')
matplotlib.pyplot.tight_layout()
print('\n              DATA:                          MODEL:                           RESIDUAL:') 
matplotlib.pyplot.show()

print('Fit Parameters:\n')
print(fit_result.amplitude)
print(fit_result.x_mean)
print(fit_result.y_mean)
print(fit_result.x_stddev)
print(fit_result.y_stddev)
print(fit_result.theta)

psf_fwhm_pix = numpy.around(numpy.mean([fit_result.x_stddev.value, fit_result.y_stddev.value])*2.354, decimals=3)

print('\nThe PSF FWHM is', psf_fwhm_pix, 'pixels.')

***

# Data Products

The code will output two PSF models. The first is a mean-stack of the subimages while the second is a median-stack. If *dual_psf = False* then the results will be output into *stellar* and *empirical* folders. If *dual_psf = True* then there will be sub-folders named "inner" and "outer" with the resulting model files.

***

# Troubleshooting <a class="anchor" id="trouble"><a>

 - **The code is taking a very long time to run, how can I speed it up?**
      - The majority of computation time is spent in the Moffat/Gaussian profile fitting, so use *parallel_mode = True* with as many cores available for a significant performance boost. Also, if you cannot lower the grid resolution (set by *interp_grid_steps*), consider fitting fewer sources. If the stars are very symmetric, which may be the case with some UVIS images, consider using the peak locations (*use_peaks = True*) instead of fitting profiles.


 - **I'm using *psf_type = empirical* with a drizzle made by *wfc3tools* and the code cannot find the planted PSFs?**
 
      - Go to this line: [*peaks_table = find_peaks(...)*](#import2) and modify the threshold value. In general this should be robust for most images. Set the threshold approximately equal to ~ 10-20% of the peak flux values in your drizzle or higher until it only locates the PSF peaks.
      
[Table of Contents](#tag0)      
      
***

# Developer Notes <a class="anchor" id="developer"><a>

**The following action items are planned for future releases:**
 - Have the code read-in a DS9 region file or weight map (rather than using a circle) for inner/outer regions.
 
 - Have the code overplot the selected stars on the mosaic for visual inspection and publication figures. 
  
 - Confirm the 'even' and 'odd' modes produce PSF models that are always centered on the output image grid.
 
 - Confirm the code works properly for drizzles produced using AstroDrizzle with the *build = True* option.
   
 - Have the code determine the *find_peaks* threshold using *from astropy.stats import sigma_clipped_stats*.
 
 - Note that models run with read_positions = T/F have the same filename and will be over-written.

[Table of Contents](#tag0)

In [ ]:
if speak == True:
    
    os.system("say 'your notebook has finished running.'")